In [1]:
import os

In [2]:
%pwd

'c:\\Users\\kanis\\Desktop\\Audio_Classification\\research'

In [3]:
os.chdir(r"c:\\Users\\kanis\\Desktop\\Audio_Classification")

In [4]:
%pwd

'c:\\Users\\kanis\\Desktop\\Audio_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig():
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
%pwd

'c:\\Users\\kanis\\Desktop\\Audio_Classification'

In [7]:
%pwd

'c:\\Users\\kanis\\Desktop\\Audio_Classification'

In [8]:
os.chdir(r"c:\\Users\\kanis\\Desktop\\Audio_Classification")

In [9]:
from Audio_Vision.utils.common import read_yaml, create_directories

class ConfigurationManager():
    def __init__(self):
        self.config_filepath = Path("config/config.yaml")
        self.params_filepath = Path("params.yaml")

        self.config = read_yaml(self.config_filepath)
        self.params = read_yaml(self.params_filepath)

        create_directories([self.config.artifacts_root]) 

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=  config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config

In [10]:
import os 
import gdown
import zipfile
from Audio_Vision import logger
from Audio_Vision.utils.common import get_size

In [11]:
class DataIngestion():
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)->str:
        ''' 
        Fetch data from the URL
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir,exist_ok = True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extracts the zip file into data directory 
        Function returns None'''

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e